# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [8]:
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'
r = requests.get(url)
r_json = r.json()

r_json_items = r_json['items']
df_r_json = pd.DataFrame(r_json_items)


In [12]:
#df_r_json.info() 
df_r_json.status.value_counts()
#deferido, arquivado e regularizado são os mais frequentes

Deferido                                                 91
Arquivado                                                53
Regularizado                                             25
Arquivado por decurso de prazo                           23
Deferido (PVL-IF)                                        16
Encaminhado à PGFN com manifestação técnica favorável    13
Arquivado a pedido                                        6
Em retificação pelo interessado                           4
Pendente de regularização                                 1
Em retificação pelo interessado (PVL-IF)                  1
Name: status, dtype: int64

In [107]:
# 3) Seu código aqui
data_serie = df_r_json.data_status
data_df = pd.DataFrame(data_serie)
data_df_split = data_df['data_status'].str.split('/')

anos = []
for i in data_df_split :
    anos.append(i[2])
    
df_anos = pd.DataFrame(anos, columns = ['Freq_anos'])
df_anos['Freq_anos'].value_counts()

2008    37
2007    29
2014    20
2010    19
2019    17
2013    15
2020    15
2012    13
2022    12
2011    10
2009     9
2015     9
2017     5
2021     5
2018     4
2002     4
2016     4
2006     4
2003     1
2004     1
Name: Freq_anos, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [136]:
def url_uf (uf, tipo):
    url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo}'
    r = requests.get(url)
    r_json = r.json()
    df_r_json = pd.DataFrame(r_json['items'])
    return df_r_json


df_MG = url_uf("MG","Estado")

df_MG['status'].value_counts()
# 1 solicitação




Encaminhado à PGFN com manifestação técnica favorável    20
Deferido                                                  7
Arquivado a pedido                                        5
Arquivado                                                 3
Arquivado pela STN                                        3
Indeferido                                                1
Arquivado por decurso de prazo                            1
Name: status, dtype: int64

In [137]:
df_BA = url_uf("BA","Município")
df_BA['interessado'].value_counts()
#Camaçari

Camaçari                  18
Salvador                  16
Luís Eduardo Magalhães    15
Vitória da Conquista      14
Amargosa                  14
                          ..
Laje                       1
Santa Cruz Cabrália        1
Fátima                     1
Varzedo                    1
Itiúba                     1
Name: interessado, Length: 285, dtype: int64

In [139]:
df_BA_estado = url_uf("BA","Estado")
df_BA_estado.to_csv("Dados_solicitacao_BA_estado.csv", sep=',', decimal=',', index = False)

In [ ]:
# 4) Seu código aqui